In [1]:
import os

import msgspec

from cffi import FFI

In [2]:
absolute_path = os.path.dirname('')

ffi = FFI()
ffi.cdef(
    open(os.path.join(absolute_path, "./matcher_c.h"), "r", encoding="utf-8").read()
)

lib = ffi.dlopen(os.path.join(absolute_path, "./matcher_c.so"))

In [3]:
matcher = lib.init_matcher(
    msgspec.msgpack.encode(
        {
            "test": [
                {
                    "table_id": 1,
                    "wordlist": ["你好"],
                    "exemption_wordlist": [],
                    "match_table_type": "simple",
                    "simple_match_type": 15,
                }
            ]
        }
    )
)

In [4]:
lib.matcher_is_match(matcher, "你好".encode("utf-8"))

True

In [5]:
res = lib.matcher_word_match(matcher, "你好".encode("utf-8"))
print(ffi.string(res).decode("utf-8"))
lib.drop_string(res)

{"test":"[{\"table_id\":1,\"word\":\"你好\"}]"}


In [6]:
lib.drop_matcher(matcher)

In [7]:
simple_matcher = lib.init_simple_matcher(
    msgspec.msgpack.encode(({15: [{"word_id": 1, "word": "你好"}]}))
)

In [8]:
lib.simple_matcher_is_match(simple_matcher, "你好".encode("utf-8"))

True

In [9]:
res = lib.simple_matcher_process(simple_matcher, b"\xe4\xbd\xa0\xe5\xa5\xbd")
print(ffi.string(res).decode("utf-8"))
lib.drop_string(res)

[{"word_id":1,"word":"你好"}]


In [10]:
lib.drop_simple_matcher(simple_matcher)